In [1]:
import numpy as np
import math
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1,28*28).astype("float32")
x_test = x_test.reshape(-1,28*28).astype("float32")
x_train = tf.keras.utils.normalize(x_train,axis=1)
x_test = tf.keras.utils.normalize(x_test,axis=1)
y_train = LabelBinarizer().fit_transform(y_train)
y_test = LabelBinarizer().fit_transform(y_test)

In [3]:
class Q2:
    
    def __init__(self, layers, batch_size, lr=0.1):
        self.W = []
        self.layers = layers
        self.batch_size = batch_size
        self.lr = lr
        
        for i in np.arange(0, len(layers) - 2):
            w = np.random.randn(layers[i]+1,layers[i+1]+1)
            self.W.append(w/np.sqrt(layers[i]))
            
        w = np.random.randn(layers[-2]+1,layers[-1])
        self.W.append(w)
        
    def sigmoid(self,x):
        return 1.0/(1.0+np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x*(1 - x)
    
    def model_prepare(self, x, y):
        arr = [np.atleast_2d(x)]
        
        for layer in np.arange(0, len(self.W)):
            net = arr[layer].dot(self.W[layer])
            out = self.sigmoid(net)
            arr.append(out)
            
        error = arr[-1] - y
        der = [error * self.sigmoid_derivative(arr[-1])]
        
        for layer in np.arange(len(arr) - 2, 0, -1):
            del_d = der[-1].dot(self.W[layer].T)
            del_d = del_d * self.sigmoid_derivative(arr[layer])
            der.append(del_d)
            
        der = der[::-1]
        
        for layer in np.arange(0, len(self.W)):
            self.W[layer] += -self.lr * arr[layer].T.dot(der[layer])
            
    def loss(self, X, targets):
        targets = np.atleast_2d(targets)
        predictions = self.predict(X, addBias=False)
        losses = 0.5 * np.sum((predictions - targets) ** 2)
        return losses
    
    def predict(self, X, addBias=True):
        pred = np.atleast_2d(X)
        if addBias:
            pred = np.c_[pred, np.ones((pred.shape[0]))]
        
        for j in np.arange(0, len(self.W)):
            pred = self.sigmoid(np.dot(pred, self.W[j]))

        return pred
    
    def train(self, X, y, epochs=100):
        X = np.c_[X, np.ones((X.shape[0]))]
        for epoch in np.arange(0, epochs):
            
            for j in range(0,X.shape[0],self.batch_size):
                x = X[j:j+self.batch_size]
                target = y[j:j+self.batch_size]
                self.model_prepare(x, target)
                
            loss = self.loss(X, y)

In [4]:
final = Q2(layers = [784, 15, 10],batch_size = 32)

final.train(x_train, y_train, epochs=5)

In [5]:
predictions = final.predict(x_test)
predictions = predictions.argmax(axis=1)
print("Accuracy :",accuracy_score(y_test.argmax(axis=1), predictions))

Accuracy of Neural network using Backpropagation-our model is : 0.9205


In [6]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

In [7]:
def loss_fun(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    loss = 0.5 * tf.reduce_sum((y_pred - y_true) ** 2)
    return loss

In [8]:
Default_net = keras.Sequential()
Default_net.add(keras.layers.InputLayer((784)))

neurons = [15,10]

initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=1.0)

size = len(neurons)
for i in range(0,size-1,1):
    Default_net.add(layers.Dense(neurons[i], activation='sigmoid', kernel_initializer=initializer, bias_initializer=initializer))

Default_net.add(layers.Dense(neurons[size-1], activation='sigmoid', kernel_initializer=initializer, use_bias = False))

Default_net.compile(
loss=loss_fun, 
optimizer=tf.keras.optimizers.SGD(learning_rate = 0.1),
metrics=["accuracy"])

Default_net.fit(x_train, y_train, batch_size=32, epochs=5, verbose=1)

Epoch 1/5
1875/1875 [==============================] - 2s 982us/step - loss: 7.3971 - accuracy: 0.6980
Epoch 2/5
1875/1875 [==============================] - 2s 989us/step - loss: 3.6714 - accuracy: 0.8735
Epoch 3/5
1875/1875 [==============================] - 2s 989us/step - loss: 2.9890 - accuracy: 0.8951
Epoch 4/5
1875/1875 [==============================] - 2s 935us/step - loss: 2.6762 - accuracy: 0.9046
Epoch 5/5
1875/1875 [==============================] - 2s 944us/step - loss: 2.4890 - accuracy: 0.9110
